In [5]:
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models
# from effnetv2 import effnetv2_s
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

# 设置全局参数
modellr = 1e-4
BATCH_SIZE = 32
EPOCHS = 1
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# 数据预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])

])
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [7]:
# 读取数据
dataset_train = datasets.ImageFolder('../data/train', transform)
# print(dataset_train.imgs)
# 对应文件夹的label
# print(dataset_train.class_to_idx)
dataset_val = datasets.ImageFolder('../data/val', transform_test)
# 对应文件夹的label
# print(dataset_test.class_to_idx)
dataset_test = datasets.ImageFolder('../data/test', transform_test)
# 导入数据
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)
val_loader = torch.utils.data.DataLoader(dataset_val,
 batch_size=BATCH_SIZE, shuffle=False)

In [8]:
# 实例化模型并且移动到GPU
criterion = nn.CrossEntropyLoss()
model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 12)
model.to(DEVICE)
# 选择简单暴力的Adam优化器，学习率调低
optimizer = optim.Adam(model.parameters(), lr=modellr)


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    modellrnew = modellr * (0.1 ** (epoch // 50))
    print("lr:", modellrnew)
    for param_group in optimizer.param_groups:
        param_group['lr'] = modellrnew

In [9]:
# 定义训练过程
writer = SummaryWriter("E:/大三下/生物大数据分析/project/code/train_log0617")
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    sum_loss = 0
    total_num = len(train_loader.dataset)
    print(total_num, len(train_loader))
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print_loss = loss.data.item()
        sum_loss += print_loss
        if (batch_idx + 1) % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                       100. * (batch_idx + 1) / len(train_loader), loss.item()))
            writer.add_scalar('Train_loss', loss.item(), epoch)
    ave_loss = sum_loss / len(train_loader)
    print('epoch:{},loss:{}'.format(epoch, ave_loss))
    writer.close()

def val(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total_num = len(test_loader.dataset)
    print(total_num, len(test_loader))
    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data).to(device), Variable(target).to(device)
            output = model(data)
            loss = criterion(output, target)
            _, pred = torch.max(output.data, 1)
            correct += torch.sum(pred == target)
            print_loss = loss.data.item()
            test_loss += print_loss
        correct = correct.data.item()
        acc = correct / total_num
        avgloss = test_loss / len(test_loader)
        print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            avgloss, correct, len(test_loader.dataset), 100 * acc))


# 训练

for epoch in range(1, EPOCHS + 1):
    adjust_learning_rate(optimizer, epoch)
    train(model, DEVICE, train_loader, optimizer, epoch)
    val(model, DEVICE, test_loader)
torch.save(model, 'model.pth')

lr: 0.0001
65000 2032
Train Epoch: 1 [1600/65000 (2%)]	Loss: 0.635715
Train Epoch: 1 [3200/65000 (5%)]	Loss: 0.544492
Train Epoch: 1 [4800/65000 (7%)]	Loss: 0.448718
Train Epoch: 1 [6400/65000 (10%)]	Loss: 0.801663
Train Epoch: 1 [8000/65000 (12%)]	Loss: 0.344649
Train Epoch: 1 [9600/65000 (15%)]	Loss: 0.332533
Train Epoch: 1 [11200/65000 (17%)]	Loss: 0.302140
Train Epoch: 1 [12800/65000 (20%)]	Loss: 0.259712
Train Epoch: 1 [14400/65000 (22%)]	Loss: 0.229060
Train Epoch: 1 [16000/65000 (25%)]	Loss: 0.252308
Train Epoch: 1 [17600/65000 (27%)]	Loss: 0.245694
Train Epoch: 1 [19200/65000 (30%)]	Loss: 0.198829
Train Epoch: 1 [20800/65000 (32%)]	Loss: 0.176436
Train Epoch: 1 [22400/65000 (34%)]	Loss: 0.263124
Train Epoch: 1 [24000/65000 (37%)]	Loss: 0.263623
Train Epoch: 1 [25600/65000 (39%)]	Loss: 0.263773
Train Epoch: 1 [27200/65000 (42%)]	Loss: 0.174454
Train Epoch: 1 [28800/65000 (44%)]	Loss: 0.152239
Train Epoch: 1 [30400/65000 (47%)]	Loss: 0.146688
Train Epoch: 1 [32000/65000 (49%)]	Lo

In [35]:
#原图片 84%
#resize 87%
#nopretrain 75%
val(model, DEVICE, val_loader)

12500 391

Val set: Average loss: 0.4108, Accuracy: 10849/12500 (87%)



In [11]:
val(model, DEVICE, test_loader)

12500 391

Val set: Average loss: 0.4243, Accuracy: 10788/12500 (86%)



In [12]:
val(model, DEVICE, train_loader)

65000 2032

Val set: Average loss: 0.2206, Accuracy: 60191/65000 (93%)

